In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json as js
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec



In [14]:
#load pickle file
df = pd.read_pickle('data/cleaned_elden_ring_data.pkl')


In [ ]:
df.tail(20)


In [65]:
with open('/Users/divygobiraj/Desktop/projects/ER_Lore_Bot/ER-lore-bot/data_processing/data/cleaned_elden_ring_data.json') as fp:
    data = js.load(fp)

In [ ]:
data

In [22]:


load_dotenv(dotenv_path="/Users/divygobiraj/Desktop/projects/ER_Lore_Bot/ER-lore-bot/.env")
# print(os.getenv("PINECONE_KEY"))

pc = Pinecone(api_key=os.getenv("PINECONE_KEY"))
index = pc.Index("er-text")


In [ ]:
def embed_text(text):
    return pc.inference.embed(
        model="multilingual-e5-large",
        inputs=[text],
        parameters={"input_type": "passage", "truncate": "END"}
    )


chunked_vectors = []
batch_size = 200
for i in range(0, len(df), 200):
    vectors = []
    keepgo = 1
    while i % 200 > 0 or keepgo == 1:
        row = df.iloc[i]
        print(i)
        keepgo = 0
        vectors.append(
            {
                "id": row['name'],
                "values": (embed_text(row['name'] + ": " +  row['text'])).data[0].values, 
                "metadata": {"type": row['type']}
            }
        )
        i+=1
    chunked_vectors.append(vectors)
    vectors = []
    keepgo = 1





    




In [ ]:
row = df.iloc[0]
for chunk in chunked_vectors[0]:
    for vector in chunk:
        #if vector['id'] is not ascii change to ascii, eg e to e
        vector['id'] = vector['id'].encode('ascii', 'ignore').decode('ascii')



In [ ]:
for vec in chunked_vectors:
    index.upsert(vectors=vec)
